In [260]:
import numpy as np
from itertools import permutations, product
from collections import Counter, defaultdict

In [254]:
# fname = 'example_short.txt'
fname = 'example.txt'
# fname = 'input.txt'
with open(fname) as f:
    lines = f.readlines()

lines

scanners = []
start, end = 0, 0
for ii, line in enumerate(lines + ["\n"]):
    if "scanner" in line:
        start = ii + 1
    elif line == "\n":
        end = ii
    if end > start:
        scanners.append(lines[start : end])

        
scanners = [[[int(num) for num in beacon.strip().split(',')] for beacon in scanner] for scanner in scanners]
# scanners
scanner_dict = dict()
for ii in range(len(scanners)):
    scanner_dict[ii] = np.array(scanners[ii])
# scanner_dict

In [255]:
len(scanner_dict), [len(scanner_dict[ii]) for ii in range(len(scanner_dict))]

(5, [25, 25, 26, 25, 26])

# Generating all valid right-hand-rule COB matrices
1. generate all axis direction vectors a, b
2. for every non-same a, b pair, find c = a cross b
3. assemble matrix [a, b, c]

In [322]:
all_basis_vectors = [
    np.array([1, 0, 0]),
    np.array([0, 1, 0]),
    np.array([0, 0, 1]),
    np.array([-1, 0, 0]),
    np.array([0, -1, 0]),
    np.array([0, 0, -1]),
]

COBs_3D = []
COBs_inv_3D = []
count = 0
for e_i in all_basis_vectors:
    for e_j in all_basis_vectors:
        if (e_i != e_j).any() and (e_i != -1 * e_j).any():
            count += 1
            e_k = np.cross(e_i, e_j)
            COB = np.vstack([e_i, e_j, e_k]).T
            COBs_3D.append(COB)
            COBs_inv_3D.append(np.linalg.inv(COB).astype(int))
assert count == 24
# COBs_inv_3D

In [188]:
all_basis_vectors = [
    np.array([1, 0, 0]),
    np.array([0, 1, 0]),
    np.array([-1, 0, 0]),
    np.array([0, -1, 0]),
]

COBs_2D = []
COBs_inv_2D = []
count = 0
for e_i in all_basis_vectors:
    for e_j in all_basis_vectors:
        if (e_i != e_j).any() and (e_i != -1 * e_j).any():
            if (np.cross(e_i, e_j) >= 0).all():
                # check that cross product is positive
                count += 1
                COB = np.vstack([e_i[:-1], e_j[:-1]]).T
                COBs_2D.append(COB)
                COBs_inv_2D.append(np.linalg.inv(COB).astype(int))
assert count == 4

In [190]:
COBs_2D, COBs_inv_2D

([array([[1, 0],
         [0, 1]]),
  array([[ 0, -1],
         [ 1,  0]]),
  array([[-1,  0],
         [ 0, -1]]),
  array([[ 0,  1],
         [-1,  0]])],
 [array([[1, 0],
         [0, 1]]),
  array([[ 0,  1],
         [-1,  0]]),
  array([[-1,  0],
         [ 0, -1]]),
  array([[ 0, -1],
         [ 1,  0]])])

In [191]:
COBs_2D[1]

array([[ 0, -1],
       [ 1,  0]])

# Test a simpler 2d case
## Setup
* $S_0$ shall be the refence coordinate system
* S_1 shall be at a $k*90$ degree orientation to $S_0$, defined by a change of basis matrix
* points will be generated wrt $S_0$ and translated to $S_1$
* all 4 possible inverse change-of-bases matrices will be attempted to get original points (or is it S_1 location?)

In [192]:
S1_COB = np.array([[0, -1], [1, 0]]) # rotate x and y 90 ccw, COBs_2D[1]
S1_pos = np.array([5, 5])
S1_COB, S1_pos

(array([[ 0, -1],
        [ 1,  0]]),
 array([5, 5]))

In [193]:
points = [
    np.array([1, 1]),
    np.array([2, 4]),
    np.array([3, 3]),
    np.array([0, 4]),
    np.array([4, 3]),
]

points_rel_S1 = [S1_COB @ (point - S1_pos) for point in points]
points_rel_S1_arr = np.array(points_rel_S1)
#points, points_rel_S1, 
points_rel_S1_arr

array([[ 4, -4],
       [ 1, -3],
       [ 2, -2],
       [ 1, -5],
       [ 2, -1]])

In [194]:
S1_COB_inv = np.linalg.inv(S1_COB).astype(int)
S1_COB @ S1_COB_inv

array([[1, 0],
       [0, 1]])

In [200]:
for inv in COBs_inv_2D:
    points_xfmd = (inv @ points_rel_S1_arr.T).T
#     print(inv)
    diffs = [px - pt for px, pt in zip(points_xfmd, points)]
    
    # count number of unique pairs (fewer means more matches)
    diffs = [tuple(diff) for diff in diffs]
    print(len(set(diffs)))
    
    print(diffs)

5
[(3, -5), (-1, -7), (-1, -5), (1, -9), (-2, -4)]
1
[(-5, -5), (-5, -5), (-5, -5), (-5, -5), (-5, -5)]
5
[(-5, 3), (-3, -1), (-5, -1), (-1, 1), (-6, -2)]
5
[(3, 3), (1, -3), (-1, -1), (5, -3), (-3, -1)]


# test with points not in order...

In [150]:
points_rel_S1_arr_scrambled = points_rel_S1_arr[[2,3,0,4,1], :]
points_rel_S1_arr_scrambled

array([[ 2, -2],
       [ 1, -5],
       [ 4, -4],
       [ 2, -1],
       [ 1, -3]])

In [214]:
inv = invs[-1] # the correct one
points_xfmd = (inv @ points_rel_S1_arr_scrambled.T).T
print(points_xfmd)
points_xfmd = [tuple(point) for point in points_xfmd]
points_xfmd = sorted(points_xfmd)
points_xfmd = np.array(points_xfmd)
print(points_xfmd)

points_temp = [tuple(point) for point in points]
points_temp = sorted(points_temp)
points_temp = np.array(points_temp)
print([px - pt for px, pt in zip(points_xfmd, points_temp)])

[[-2 -2]
 [-5 -1]
 [-4 -4]
 [-1 -2]
 [-3 -1]]
[[-5 -1]
 [-4 -4]
 [-3 -1]
 [-2 -2]
 [-1 -2]]
[array([-5, -5]), array([-5, -5]), array([-5, -5]), array([-5, -5]), array([-5, -5])]


## subtract each point from every other point ?


In [227]:
stuff = []
for px, pt in product(points_xfmd, points_temp):
    diff = tuple(px - pt)
    stuff.append(diff)
#     print(px, pt, diff)

max(Counter(stuff).items(), key = lambda pair: pair[1])

((-5, -5), 5)

In [228]:
for inv in COBs_inv_2D:
    points_xfmd = (inv @ points_rel_S1_arr.T).T
    stuff = []
    for px, pt in product(points_xfmd, points_temp):
        diff = tuple(px - pt)
        stuff.append(diff)
    #     print(px, pt, diff)

    print(max(Counter(stuff).items(), key = lambda pair: pair[1]))

((1, -7), 2)
((-5, -5), 5)
((-4, 0), 2)
((4, 0), 2)


## ^ this example is successful at least ... try the small demo example?

In [288]:
def find_relative_difference(reference_pts, relative_pts, print_stuff=False):
    d_vects = []
    for inv in COBs_inv_3D:
        for p_rel, p_ref in product((inv @ relative_pts.T).T, reference_pts):
            d_vect = tuple(p_rel - p_ref)
            d_vects.append(d_vect)
    if print_stuff:
        print(sorted(Counter(d_vects).items(), key=lambda pair: -1 * pair[1])[0:10])
    location, matches = max(Counter(d_vects).items(), key=lambda pair: pair[1])
    location = np.array(location) * -1
    return matches, location

find_relative_difference(scanner_dict[0], scanner_dict[1]), find_relative_difference(scanner_dict[1], scanner_dict[0])

((12, array([   68., -1246.,   -43.])), (12, array([  68., 1246.,  -43.])))

In [314]:
def find_relative_difference_with_inv(reference_pts, relative_pts, print_stuff=False):
    d_vects = []
    for inv_idx, inv in enumerate(COBs_inv_3D):
        for p_rel, p_ref in product((inv @ relative_pts.T).T, reference_pts):
            d_vect = tuple(p_ref - p_rel)
            d_vects.append((d_vect, inv_idx))
    if print_stuff:
        print(sorted(Counter(d_vects).items(), key=lambda pair: -1 * pair[1])[0:10])
    (location, inv_idx), matches = max(Counter(d_vects).items(), key=lambda pair: pair[1])
#     location = np.array(location) * -1
    return matches, location, inv_idx

find_relative_difference_with_inv(scanner_dict[0], scanner_dict[1], print_stuff=True), find_relative_difference_with_inv(scanner_dict[1], scanner_dict[0], print_stuff=True)

[(((68, -1246, -43), 12), 12), (((1147, -167, -438), 16), 2), (((-282, -1010, -1479), 0), 1), (((-158, -1065, -169), 0), 1), (((-1524, 169, 156), 0), 1), (((-296, -1097, -1371), 0), 1), (((-1223, -1245, -1036), 0), 1), (((-1171, -779, -231), 0), 1), (((-1031, -733, -197), 0), 1), (((-1347, -1238, -1153), 0), 1)]
[(((68, 1246, -43), 12), 12), (((167, 1147, 438), 6), 2), (((282, 1010, 1479), 0), 1), (((201, 1011, 1316), 0), 1), (((111, 1505, 540), 0), 1), (((-740, 1246, 1759), 0), 1), (((-309, 726, 923), 0), 1), (((-880, 1207, 1748), 0), 1), (((-744, 19, 55), 0), 1), (((163, 227, 1628), 0), 1)]


((12, (68, -1246, -43), 12), (12, (68, 1246, -43), 12))

In [315]:
find_relative_difference_with_inv(scanner_dict[4], scanner_dict[1], print_stuff=True), find_relative_difference_with_inv(scanner_dict[1], scanner_dict[4], print_stuff=True)

[(((-1104, -88, 113), 20), 12), (((41, 170, -16), 0), 1), (((-979, -976, 201), 0), 1), (((-245, 189, -1039), 0), 1), (((-1400, 43, -1354), 0), 1), (((-1429, 5, -1382), 0), 1), (((-1346, -901, -1004), 0), 1), (((146, -1054, -118), 0), 1), (((241, -907, -1016), 0), 1), (((-278, -29, -1084), 0), 1)]
[(((88, 113, -1104), 7), 12), (((-41, -170, 16), 0), 1), (((-122, -169, -147), 0), 1), (((-212, 325, -923), 0), 1), (((-1063, 66, 296), 0), 1), (((-632, -454, -540), 0), 1), (((-1203, 27, 285), 0), 1), (((-1067, -1161, -1408), 0), 1), (((-160, -953, 165), 0), 1), (((-1187, 11, -1014), 0), 1)]


((12, (-1104, -88, 113), 20), (12, (88, 113, -1104), 7))

In [316]:
connections = defaultdict(list)
offsets = dict()
inv_idxs = dict()
for ii in range(len(scanner_dict)):
    for jj in range(len(scanner_dict)):
        if ii == jj:
            continue
#         matches, location = find_relative_difference(scanner_dict[ii], scanner_dict[jj])
        matches, location, inv_idx = find_relative_difference_with_inv(scanner_dict[ii], scanner_dict[jj])
        if matches >= 12:
#             print(ii, jj, matches, location)
            print(ii, jj, matches, location, inv_idx)
            # the 'ii' one is the reference
            connections[ii].append(jj)
            offsets[(ii, jj)] = location
            inv_idxs[(ii, jj)] = inv_idx

print(connections)
print(offsets)
print(inv_idxs)

0 1 12 (68, -1246, -43) 12
1 0 12 (68, 1246, -43) 12
1 3 12 (160, -1134, -23) 0
1 4 12 (88, 113, -1104) 7
2 4 12 (1125, -168, 72) 4
3 1 12 (-160, 1134, 23) 0
4 1 12 (-1104, -88, 113) 20
4 2 12 (168, -1125, 72) 4
defaultdict(<class 'list'>, {0: [1], 1: [0, 3, 4], 2: [4], 3: [1], 4: [1, 2]})
{(0, 1): (68, -1246, -43), (1, 0): (68, 1246, -43), (1, 3): (160, -1134, -23), (1, 4): (88, 113, -1104), (2, 4): (1125, -168, 72), (3, 1): (-160, 1134, 23), (4, 1): (-1104, -88, 113), (4, 2): (168, -1125, 72)}
{(0, 1): 12, (1, 0): 12, (1, 3): 0, (1, 4): 7, (2, 4): 4, (3, 1): 0, (4, 1): 20, (4, 2): 4}


In [323]:
def traverse_connections(conn, offsets, inv_idxs, node=0, current_offset=np.array([0, 0, 0]), already_visited=set()):
    already_visited.add(node)
    to_visit = set(conn[node]).difference(already_visited)
    if len(to_visit) == 0:
        return []
    for child_node in to_visit:
        print(f"from {node} visit {child_node}")
        offset = current_offset + offsets[(node, child_node)] #( COBs_inv_3D[inv_idxs[(node, child_node)]] @ offsets[(node, child_node)] )
        print(offset)
        list_of_offsets = traverse_connections(conn, offsets, inv_idxs, child_node, offset, already_visited)
        
traverse_connections(connections, offsets, inv_idxs)

from 0 visit 1
[   68 -1246   -43]
from 1 visit 3
[  228 -2380   -66]
from 1 visit 4
[  156 -1133 -1147]
from 4 visit 2
[  324 -2258 -1075]


In [313]:
COBs_inv_3D[inv_idxs[(0, 1)]]

array([[-1,  0,  0],
       [ 0,  1,  0],
       [ 0,  0, -1]])

# Algorithm for the challenge
## steps
0. compute all 24 possible inverse COB matrices
1. for each sensor $S_i$
2. transform each point with all iCOB matrices
    * perhaps sort the points, then slide one array over the other, differencing the overlap?
3. subtract all transformed points from $S_0$ points
    * **This doesn't work, as the point order may be different. Can the points be sorted?**
4. if enough are the same, this gives the location of $S_0$ relative to $S_i$

...

repeat for other sensors.

**how to handle offsetting, since not all points are in common?**

In [ ]:
# assume given matrices, scanner info
sp = find_scanner_pairs()
find_abs_positions(sp)